In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,160 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://security.ubuntu.co

In [ ]:
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# disable img bc there are alot
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(180)
wait = WebDriverWait(driver, 15)

# check if cookies are accepted
cookies_accepted = False

def accept_cookies():
    global cookies_accepted
    if not cookies_accepted:
        try:
            cookie_button = wait.until(EC.element_to_be_clickable((By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll')))
            cookie_button.click()
            cookies_accepted = True
            print("Cookies accepted.")
        except Exception as e:
            print(f"Error accepting cookies: {e}")

def get_article_links(search_term, page_num=1):
    base_url = f"https://politiken.dk/search/?q={search_term}&sort=pd&page={page_num}"
    driver.get(base_url)
    accept_cookies()
    time.sleep(3)

    article_elements = driver.find_elements(By.CSS_SELECTOR, 'div.search-result__article')
    article_links = []

    for article in article_elements:
        try:
            link_element = article.find_element(By.TAG_NAME, 'a')
            article_url = link_element.get_attribute('href')
            article_links.append(article_url)
        except Exception as e:
            print(f"Error finding article link: {e}")

    return article_links

def login_to_politiken():
    driver.get('https://medielogin.dk/politiken/login')
    try:
        email_input = wait.until(EC.element_to_be_clickable((By.ID, 'Username')))
        password_input = driver.find_element(By.ID, 'Password')

        email_input.send_keys('maria.norgreen@gmail.com')
        password_input.send_keys('Nalle2039')

        submit_button = driver.find_element(By.CSS_SELECTOR, 'button.ml-submit')
        submit_button.click()
        print("Logged in successfully.")
        time.sleep(2)
    except Exception as e:
        print(f"Login failed: {e}")

def check_and_login(article_url=None):
    try:
        log_ind_button = wait.until(EC.presence_of_element_located((By.LINK_TEXT, 'Log ind')))
        if log_ind_button:
            print("Log ind button found. Logging in...")
            log_ind_button.click()
            login_to_politiken()
            time.sleep(3)

            if article_url:
                print(f"Revisiting article after login: {article_url}")
                driver.get(article_url)
                time.sleep(2)

        else:
            print("Log ind button not found, no login required.")
    except TimeoutException:
        print("Log ind button not present, continuing without login.")

def scrape_article(article_url):
    driver.get(article_url)
    time.sleep(3)

    check_and_login(article_url=article_url)

    try:
        title = driver.find_element(By.CSS_SELECTOR, 'h1.font-serif-header-96').text
    except NoSuchElementException:
        title = 'No title found'

    try:
        subheader = driver.find_element(By.CSS_SELECTOR, 'p.tagline').text
    except NoSuchElementException:
        subheader = 'No subheader'

    try:
        datetime = driver.find_element(By.CSS_SELECTOR, 'p.font-label-14').text
    except NoSuchElementException:
        datetime = 'No datetime'

    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, 'p')
        content = " ".join([p.text for p in paragraphs])
    except NoSuchElementException:
        content = 'No content found'

    return {
        'url': article_url,
        'title': title,
        'subheader': subheader,
        'datetime': datetime,
        'content': content
    }

def scrape_politiken_articles(search_terms, max_pages=None):
    articles_data = []

    login_to_politiken()  #  start by login in

    for search_term in search_terms:
        page = 1

        while True:
            print(f"Scraping page {page} for search term '{search_term}'...")

            article_links = get_article_links(search_term, page_num=page)
            if not article_links:
                print(f"No more articles found for search term '{search_term}'.")
                break

            for article_url in article_links:
                print(f"Scraping article: {article_url}")
                article_data = scrape_article(article_url)
                article_data['search_term'] = search_term
                articles_data.append(article_data)

            if max_pages and page >= max_pages:
                print(f"Reached max_pages limit ({max_pages}) for search term '{search_term}'.")
                break

            page += 1

    return pd.DataFrame(articles_data)

# Example usage
search_terms = ['ansigtsgenkendelse', 'facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']


df = scrape_politiken_articles(search_terms)
df.to_csv('politiken_articles_2.csv', index=False)
print(df.head())


Logged in successfully.
Scraping page 1 for search term 'sikkerhedspakke overvågning'...
Cookies accepted.
Scraping article: https://politiken.dk/search/art9360992/I-53-byer-har-politiet-f%C3%A5et-nye-overv%C3%A5gningskameraer.-Deres-placering-rejser-debat
Log ind button found. Logging in...
Logged in successfully.
Revisiting article after login: https://politiken.dk/search/art9360992/I-53-byer-har-politiet-f%C3%A5et-nye-overv%C3%A5gningskameraer.-Deres-placering-rejser-debat
Scraping article: https://politiken.dk/search/art8048102/2020-var-et-historisk-ringe-%C3%A5r-for-retssikkerheden-i-Danmark
Log ind button found. Logging in...
Logged in successfully.
Revisiting article after login: https://politiken.dk/search/art8048102/2020-var-et-historisk-ringe-%C3%A5r-for-retssikkerheden-i-Danmark
Scraping article: https://politiken.dk/search/art7739341/Skjul-dine-data-og-d%C3%B8
Log ind button found. Logging in...
Logged in successfully.
Revisiting article after login: https://politiken.dk/se

In [ ]:
df

,url,title,subheader,datetime,content,search_term
0,https://politiken.dk/search/art9360992/I-53-by...,No title found,No subheader,No datetime,Fra 2018 til 2022 har 12 politikre...,sikkerhedspakke overvågning
1,https://politiken.dk/search/art8048102/2020-va...,No title found,2020 har været et historisk ringe år for borge...,"29. DECEMBER 2020, 13.45","29. DECEMBER 2020, 13.45 DEL ARTIKLEN ...",sikkerhedspakke overvågning
2,https://politiken.dk/search/art7739341/Skjul-d...,No title found,Er privatlivet vigtigere end at undgå et måske...,"4. APRIL 2020, 12.50","4. APRIL 2020, 12.50 DEL ARTIKLEN Er...",sikkerhedspakke overvågning
3,https://politiken.dk/search/art7479720/Vi-risi...,No title found,For nylig lancerede den danske regering en sik...,"11. NOVEMBER 2019, 08.40","11. NOVEMBER 2019, 08.40 DEL ARTIKLEN ...",sikkerhedspakke overvågning
4,https://politiken.dk/search/art7457018/Masseov...,No title found,"Teknologien kan være en kæmpe fordel, siger Kø...","24. OKTOBER 2019, 19.14","24. OKTOBER 2019, 19.14 DEL ARTIKLEN ...",sikkerhedspakke overvågning
5,https://politiken.dk/search/art7432803/Regerin...,No title found,Flere tiltag skal dæmme op for den hårde krimi...,"10. OKTOBER 2019, 09.00","10. OKTOBER 2019, 09.00 DEL ARTIKLEN ...",sikkerhedspakke overvågning
6,https://politiken.dk/search/art7432758/Regerin...,No title found,No subheader,"10. OKTOBER 2019, 06.56","10. OKTOBER 2019, 06.56 DEL ARTIKLEN ...",sikkerhedspakke overvågning
7,https://politiken.dk/search/art7431376/Overv%C...,No title found,Regeringen vil komme den voldelige kriminalite...,"9. OKTOBER 2019, 23.04","9. OKTOBER 2019, 23.04 DEL ARTIKLEN ...",sikkerhedspakke overvågning
8,https://politiken.dk/search/art7431540/Rigspol...,No title found,Data om uskyldige kan være blevet gemt i strid...,"9. OKTOBER 2019, 21.30","9. OKTOBER 2019, 21.30 DEL ARTIKLEN ...",sikkerhedspakke overvågning
9,https://politiken.dk/search/art7430938/T%C3%A6...,No title found,No subheader,"8. OKTOBER 2019, 23.00","8. OKTOBER 2019, 23.00 DEL ARTIKLEN ...",sikkerhedspakke overvågning


Did not manage to scrape title and sometimes not even subheader, as the website is changing ALOT dependning on each article --> sometimes Heading fills alot of the articles, other times its formatted in a way so it looks like an old paper, etc.

In [ ]:
df['content'][1]

'29. DECEMBER 2020, 13.45'

In [ ]:
df.drop_duplicates(subset=['url'], inplace=True)



In [ ]:
# fixing date
def clean_datetime(datetime_str):
    if 'No datetime' in datetime_str:
        return None
    cleaned_date = re.sub(r',.*', '', datetime_str)
    return cleaned_date.strip()

df['datetime'] = df['datetime'].apply(clean_datetime)

month_mapping = {
    'januar': 'January', 'februar': 'February', 'marts': 'March',
    'april': 'April', 'maj': 'May', 'juni': 'June',
    'juli': 'July', 'august': 'August', 'september': 'September',
    'oktober': 'October', 'november': 'November', 'december': 'December'
}

for danish_month, english_month in month_mapping.items():
    df['datetime'] = df['datetime'].str.replace(danish_month, english_month)

df['date'] = pd.to_datetime(df['datetime'], format='%d. %B %Y', dayfirst=True, errors='coerce')
df = df.drop(columns=['datetime'])
df

,url,title,subheader,datetime,content,search_term
0,https://politiken.dk/search/art10081658/Ja-det...,"Studerende: Ja, det bliver pisseirriterende, m...",Overvågningsteknologi kan på sigt komme til at...,26. SEPTEMBER 2024,"26. SEPTEMBER 2024, 19.59 DEBATINDLÆG A...",ansigtsgenkendelse
1,https://politiken.dk/search/art10062236/Ansigt...,No title found,Hør dagens afsnit af ’Du lytter til Politiken’.,11. SEPTEMBER 2024,"11. SEPTEMBER 2024, 04.30 Hør dagens af...",ansigtsgenkendelse
2,https://politiken.dk/search/art10061325/Datati...,No title found,Politiet har for nylig fået tilladelse til at ...,9. SEPTEMBER 2024,"9. SEPTEMBER 2024, 18.14 Politiet har f...",ansigtsgenkendelse
3,https://politiken.dk/search/art10053374/Roboth...,No title found,No subheader,None,R\nobothunde tager et nyt skridt m...,ansigtsgenkendelse
4,https://politiken.dk/search/art10057714/Politi...,No title found,"Eksperter kritiserer, at Folketinget har beslu...",6. SEPTEMBER 2024,"6. SEPTEMBER 2024, 20.12 Eksperter krit...",ansigtsgenkendelse
...,...,...,...,...,...,...
405,https://politiken.dk/search/art4892573/Overv%C...,No title found,No subheader,8. SEPTEMBER 2004,"8. SEPTEMBER 2004, 10.55 Videoovervågni...",overvågningssamfund kamera
406,https://politiken.dk/search/art5690005/Overv%C...,No title found,No subheader,8. SEPTEMBER 2004,"8. SEPTEMBER 2004, 10.55 Videoovervågni...",overvågningssamfund kamera
407,https://politiken.dk/search/art4899409/Begr%C3...,No title found,No subheader,28. APRIL 2004,"28. APRIL 2004, 22.10 Automatisk trafik...",overvågningssamfund kamera
409,https://politiken.dk/search/art5054072/The-Beg...,The Beginning of History,No subheader,2. APRIL 2012,"2. APRIL 2012, 00.01 In the 1980s my wi...",face detect


In [ ]:

def merge_content(row):
    header = '' if row['title'] == 'No title found' else row['title']
    subheader = '' if row['subheader'] == 'No subheader' else row['subheader']
    content = '' if row['content'] == 'No content found' else row['content']

    merged = ' '.join(filter(None, [header, subheader, content]))

    return merged.strip()

df['merged_content'] = df.apply(merge_content, axis=1)

df


,url,title,subheader,datetime,content,search_term,merged_content
0,https://politiken.dk/search/art9360992/I-53-by...,No title found,No subheader,No datetime,Fra 2018 til 2022 har 12 politikre...,sikkerhedspakke overvågning,Fra 2018 til 2022 har 12 politikredse sat 450 ...
1,https://politiken.dk/search/art8048102/2020-va...,No title found,2020 har været et historisk ringe år for borge...,"29. DECEMBER 2020, 13.45",2020 har været et historisk ringe år for borge...,sikkerhedspakke overvågning,2020 har været et historisk ringe år for borge...
2,https://politiken.dk/search/art7739341/Skjul-d...,No title found,Er privatlivet vigtigere end at undgå et måske...,"4. APRIL 2020, 12.50",Er privatlivet vigtigere end at undgå et måske...,sikkerhedspakke overvågning,Er privatlivet vigtigere end at undgå et måske...
3,https://politiken.dk/search/art7479720/Vi-risi...,No title found,For nylig lancerede den danske regering en sik...,"11. NOVEMBER 2019, 08.40",For nylig lancerede den danske regering en sik...,sikkerhedspakke overvågning,For nylig lancerede den danske regering en sik...
4,https://politiken.dk/search/art7457018/Masseov...,No title found,"Teknologien kan være en kæmpe fordel, siger Kø...","24. OKTOBER 2019, 19.14","Teknologien kan være en kæmpe fordel, siger Kø...",sikkerhedspakke overvågning,"Teknologien kan være en kæmpe fordel, siger Kø..."
5,https://politiken.dk/search/art7432803/Regerin...,No title found,Flere tiltag skal dæmme op for den hårde krimi...,"10. OKTOBER 2019, 09.00",Flere tiltag skal dæmme op for den hårde krimi...,sikkerhedspakke overvågning,Flere tiltag skal dæmme op for den hårde krimi...
6,https://politiken.dk/search/art7432758/Regerin...,No title found,No subheader,"10. OKTOBER 2019, 06.56",Står det til regeringen skal sprængninger ved ...,sikkerhedspakke overvågning,Står det til regeringen skal sprængninger ved ...
7,https://politiken.dk/search/art7431376/Overv%C...,No title found,Regeringen vil komme den voldelige kriminalite...,"9. OKTOBER 2019, 23.04",Regeringen vil komme den voldelige kriminalite...,sikkerhedspakke overvågning,Regeringen vil komme den voldelige kriminalite...
8,https://politiken.dk/search/art7431540/Rigspol...,No title found,Data om uskyldige kan være blevet gemt i strid...,"9. OKTOBER 2019, 21.30",Data om uskyldige kan være blevet gemt i strid...,sikkerhedspakke overvågning,Data om uskyldige kan være blevet gemt i strid...
9,https://politiken.dk/search/art7430938/T%C3%A6...,No title found,No subheader,"8. OKTOBER 2019, 23.00","Det er besynderlig timing, når statsminister M...",sikkerhedspakke overvågning,"Det er besynderlig timing, når statsminister M..."


In [ ]:
from google.colab import files

df.to_csv('Politikken_collected.csv', index=False)
files.download('Politikken_collected.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>